In [1]:
import numpy as np
import os, sys
import math
import csv
import pandas as pd
from datetime import date
import holidays

csv_file = "train.csv"
csv_data = pd.read_csv(csv_file, low_memory = False)#防止弹出警告
df = pd.DataFrame(csv_data)
df['date']


0           1/1/2017 0:00
1           1/1/2017 1:00
2           1/1/2017 2:00
3           1/1/2017 3:00
4           1/1/2017 4:00
               ...       
14001    31/12/2018 12:00
14002    31/12/2018 15:00
14003    31/12/2018 16:00
14004    31/12/2018 18:00
14005    31/12/2018 20:00
Name: date, Length: 14006, dtype: object

In [2]:
### add day / hour /  year /  month

In [3]:
temp = pd.DatetimeIndex(df['date'])
df['Day'] = temp.day
#df['Time'] = temp.time
df['hourss'] = temp.hour
df['year'] = temp.year
df['month'] = temp.month


In [4]:
df

,id,date,speed,Day,hourss,year,month
0,0,1/1/2017 0:00,43.002930,1,0,2017,1
1,1,1/1/2017 1:00,46.118696,1,1,2017,1
2,2,1/1/2017 2:00,44.294158,1,2,2017,1
3,3,1/1/2017 3:00,41.067468,1,3,2017,1
4,4,1/1/2017 4:00,46.448653,1,4,2017,1
...,...,...,...,...,...,...,...
14001,14001,31/12/2018 12:00,19.865269,31,12,2018,12
14002,14002,31/12/2018 15:00,17.820375,31,15,2018,12
14003,14003,31/12/2018 16:00,12.501851,31,16,2018,12
14004,14004,31/12/2018 18:00,15.979319,31,18,2018,12


In [5]:
df['standard_date'] = pd.to_datetime(df['date'])
df['weekday'] = df['standard_date'].dt.dayofweek+1
df['if_weekends'] = df['weekday'].apply(lambda x:0 if x<6 else 1)
#df['if_day'] = df['hourss'].apply(lambda x:1 if x>5 and x<21 else 0)
df

,id,date,speed,Day,hourss,year,month,standard_date,weekday,if_weekends
0,0,1/1/2017 0:00,43.002930,1,0,2017,1,2017-01-01 00:00:00,7,1
1,1,1/1/2017 1:00,46.118696,1,1,2017,1,2017-01-01 01:00:00,7,1
2,2,1/1/2017 2:00,44.294158,1,2,2017,1,2017-01-01 02:00:00,7,1
3,3,1/1/2017 3:00,41.067468,1,3,2017,1,2017-01-01 03:00:00,7,1
4,4,1/1/2017 4:00,46.448653,1,4,2017,1,2017-01-01 04:00:00,7,1
...,...,...,...,...,...,...,...,...,...,...
14001,14001,31/12/2018 12:00,19.865269,31,12,2018,12,2018-12-31 12:00:00,1,0
14002,14002,31/12/2018 15:00,17.820375,31,15,2018,12,2018-12-31 15:00:00,1,0
14003,14003,31/12/2018 16:00,12.501851,31,16,2018,12,2018-12-31 16:00:00,1,0
14004,14004,31/12/2018 18:00,15.979319,31,18,2018,12,2018-12-31 18:00:00,1,0


In [6]:
"""
df.drop(['Time'],axis=1,inplace=True)
df.drop(['Date'],axis=1,inplace=True)
df
"""

"\ndf.drop(['Time'],axis=1,inplace=True)\ndf.drop(['Date'],axis=1,inplace=True)\ndf\n"

In [7]:
train_y = df['speed'].values
train_y

array([43.00293016, 46.11869568, 44.29415804, ..., 12.50185109,
       15.97931873, 40.59418269])

In [8]:
#df_x = [df['hourss'].values,df['month'].values,df['year'].values,df['Day'].values]
df_x = df.values
#type(df_x)   #numpy.ndarray

train_x = np.column_stack((df_x[:,3:7],df_x[:,8:]))
#train_x = df_x[:,3:]
train_x

array([[1, 0, 2017, 1, 7, 1],
       [1, 1, 2017, 1, 7, 1],
       [1, 2, 2017, 1, 7, 1],
       ...,
       [31, 16, 2018, 12, 1, 0],
       [31, 18, 2018, 12, 1, 0],
       [31, 20, 2018, 12, 1, 0]], dtype=object)

In [9]:
from sklearn.ensemble import RandomForestRegressor #随机森林
rfc = RandomForestRegressor(n_estimators=300,random_state=90)
rfc.fit(train_x, train_y)
score_r = rfc.score(train_x, train_y)
print("Random Forest:{}".format(score_r))

Random Forest:0.9847737067935824


In [10]:

test_file = "test.csv"
test_csv = pd.read_csv(test_file, low_memory = False)#防止弹出警告
df2 = pd.DataFrame(test_csv)
temp2 = pd.DatetimeIndex(df2['date'])
df2['predict_speed'] = 0
df2['Day'] = temp2.day
#df['Time'] = temp.time
df2['hourss'] = temp2.hour
df2['year'] = temp2.year
df2['month'] = temp2.month
df2['date'] = pd.to_datetime(df2['date'])
df2['standard_date'] = pd.to_datetime(df['date'])
df2['weekday'] = df2['date'].dt.dayofweek+1
df2['if_weekends'] = df2['weekday'].apply(lambda x:0 if x<6 else 1)
#df2['if_day'] = df['hourss'].apply(lambda x:1 if x>5 and x<21 else 0)
df2

,id,date,predict_speed,Day,hourss,year,month,standard_date,weekday,if_weekends
0,0,2018-01-01 02:00:00,0,1,2,2018,1,2017-01-01 00:00:00,1,0
1,1,2018-01-01 05:00:00,0,1,5,2018,1,2017-01-01 01:00:00,1,0
2,2,2018-01-01 07:00:00,0,1,7,2018,1,2017-01-01 02:00:00,1,0
3,3,2018-01-01 08:00:00,0,1,8,2018,1,2017-01-01 03:00:00,1,0
4,4,2018-01-01 10:00:00,0,1,10,2018,1,2017-01-01 04:00:00,1,0
...,...,...,...,...,...,...,...,...,...,...
3499,3499,2018-12-31 17:00:00,0,31,17,2018,12,2017-05-27 04:00:00,1,0
3500,3500,2018-12-31 19:00:00,0,31,19,2018,12,2017-05-27 05:00:00,1,0
3501,3501,2018-12-31 21:00:00,0,31,21,2018,12,2017-05-27 06:00:00,1,0
3502,3502,2018-12-31 22:00:00,0,31,22,2018,12,2017-05-27 07:00:00,1,0


In [11]:
df2_x = df2.values

In [12]:
test_x = np.column_stack((df2_x[:,3:7],df2_x[:,8:]))
test_x

array([[1, 2, 2018, 1, 1, 0],
       [1, 5, 2018, 1, 1, 0],
       [1, 7, 2018, 1, 1, 0],
       ...,
       [31, 21, 2018, 12, 1, 0],
       [31, 22, 2018, 12, 1, 0],
       [31, 23, 2018, 12, 1, 0]], dtype=object)

In [13]:

df2

,id,date,predict_speed,Day,hourss,year,month,standard_date,weekday,if_weekends
0,0,2018-01-01 02:00:00,0,1,2,2018,1,2017-01-01 00:00:00,1,0
1,1,2018-01-01 05:00:00,0,1,5,2018,1,2017-01-01 01:00:00,1,0
2,2,2018-01-01 07:00:00,0,1,7,2018,1,2017-01-01 02:00:00,1,0
3,3,2018-01-01 08:00:00,0,1,8,2018,1,2017-01-01 03:00:00,1,0
4,4,2018-01-01 10:00:00,0,1,10,2018,1,2017-01-01 04:00:00,1,0
...,...,...,...,...,...,...,...,...,...,...
3499,3499,2018-12-31 17:00:00,0,31,17,2018,12,2017-05-27 04:00:00,1,0
3500,3500,2018-12-31 19:00:00,0,31,19,2018,12,2017-05-27 05:00:00,1,0
3501,3501,2018-12-31 21:00:00,0,31,21,2018,12,2017-05-27 06:00:00,1,0
3502,3502,2018-12-31 22:00:00,0,31,22,2018,12,2017-05-27 07:00:00,1,0


In [14]:
Ypredict = rfc.predict(test_x)

In [15]:
Ypredict

array([47.63136755, 47.01116306, 37.7736643 , ..., 44.51278428,
       39.00536677, 41.5042817 ])

In [16]:
df2['predict_speed'] = Ypredict
df2

,id,date,predict_speed,Day,hourss,year,month,standard_date,weekday,if_weekends
0,0,2018-01-01 02:00:00,47.631368,1,2,2018,1,2017-01-01 00:00:00,1,0
1,1,2018-01-01 05:00:00,47.011163,1,5,2018,1,2017-01-01 01:00:00,1,0
2,2,2018-01-01 07:00:00,37.773664,1,7,2018,1,2017-01-01 02:00:00,1,0
3,3,2018-01-01 08:00:00,28.467706,1,8,2018,1,2017-01-01 03:00:00,1,0
4,4,2018-01-01 10:00:00,34.744594,1,10,2018,1,2017-01-01 04:00:00,1,0
...,...,...,...,...,...,...,...,...,...,...
3499,3499,2018-12-31 17:00:00,16.303384,31,17,2018,12,2017-05-27 04:00:00,1,0
3500,3500,2018-12-31 19:00:00,25.029862,31,19,2018,12,2017-05-27 05:00:00,1,0
3501,3501,2018-12-31 21:00:00,44.512784,31,21,2018,12,2017-05-27 06:00:00,1,0
3502,3502,2018-12-31 22:00:00,39.005367,31,22,2018,12,2017-05-27 07:00:00,1,0


In [17]:
#df2.to_csv('test2.csv')

In [18]:
df_sub = pd.DataFrame({'id':df2['id'].values,'speed':df2['predict_speed'].values})
df_sub


,id,speed
0,0,47.631368
1,1,47.011163
2,2,37.773664
3,3,28.467706
4,4,34.744594
...,...,...
3499,3499,16.303384
3500,3500,25.029862
3501,3501,44.512784
3502,3502,39.005367


In [19]:
df_sub.to_csv('result.csv',index=False)